In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from utils import *

In [2]:
log_fnc = lambda window: -np.log(np.arange(1, window + 1) * (1 / window))
exp_fnc = lambda window: np.exp(np.arange(1, window + 1) * (1 / window))
id_fnc = lambda window: np.arange(1, window + 1) * (1 / window)
quad_fnc = lambda window: np.power(np.arange(1, window + 1) * (1 / window),2)

In [4]:
jour = 102
window = 5
df = pd.read_csv(f'data/demande/Demande-{jour}.csv', delimiter=';')
Ma_df = moving_average_growth_rate(df, window=3, column='Demand')["MA"][window:].to_numpy()
log_fnc_pred = weighted_moving_average_rate(df, window=window, column='Demand', security_rate=1, func=log_fnc)["WMA"][window:].to_numpy()
exp_fnc_pred = weighted_moving_average_rate(df, window=window, column='Demand', security_rate=1, func=exp_fnc)["WMA"][window:].to_numpy()
id_fnc_pred = weighted_moving_average_rate(df, window=window, column='Demand', security_rate=1, func=id_fnc)["WMA"][window:].to_numpy()
quad_fnc_pred = weighted_moving_average_rate(df, window=window, column='Demand', security_rate=1, func=quad_fnc)["WMA"][window:].to_numpy()
y = df["Demand"][window:].to_numpy()

In [12]:
preds = [log_fnc_pred, exp_fnc_pred, id_fnc_pred, quad_fnc_pred, Ma_df]
pred_name = ["log", "exp", "id", "quad", "MA"]
# plot the mean squared error for each prediction as a bar char in x axis pred name and y axis the mean squared error
mse = []
for pred in preds:
    mse.append(mean_absolute_error(y, pred))

# use plotly express
fig = px.bar(x=pred_name, y=mse)
fig.update_layout(
    title="Mean absolute error for each prediction using moving window of 5 days and 102 days of data",
    xaxis_title="Prediction",
    yaxis_title="Mean squared error",
    template='plotly_white'
)
# change the bar color to sns pallette 
fig.update_traces(marker_color="#4C72B0")
fig.update_traces(texttemplate='%{y:.2f}', textposition='outside')
fig.show()

In [13]:
jour = 344
df = pd.read_csv(f'data/demande/Demande-{jour}.csv', delimiter=';')


In [14]:
tmp_df = moving_average_growth_rate(df, window=3, column='Demand')
tmp_df = weighted_moving_average_rate(tmp_df, window=3, column='Demand', security_rate=1, func=quad_fnc)

In [16]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=tmp_df['Day'], y=tmp_df['Demand'], name='Demand', line=dict(color='royalblue', width=2)))
fig.add_trace(go.Scatter(x=tmp_df['Day'], y=tmp_df['MA'], name='MA', line=dict(color='firebrick', width=2)))
fig.add_trace(go.Scatter(x=tmp_df['Day'], y=tmp_df['WMA'], name='WMA', line=dict(color='green', width=2)))

number_of_days_to_forcast = 2
forcast_next_n_days = [tmp_df.iloc[-1]['Demand']] + forcast_n_nexth_days(tmp_df, number_of_days_to_forcast, 'MA')
current_max_dayx = tmp_df['Day'].max()
days = [current_max_dayx + i for i in range(0, number_of_days_to_forcast + 1)]

fig.add_trace(go.Scatter(x=days, y=forcast_next_n_days, name='Forcast', line=dict(color='indigo', width=2)))
fig.update_layout(template='plotly_white', title=f'Demande-{jour}', xaxis_title='Day', yaxis_title='Demand')

fig.show()

In [18]:
y_pred = tmp_df["WMA"][window:].to_numpy()
y = tmp_df["Demand"][window:].to_numpy()
mean_absolute_error(y, y_pred)

1.1970514950166113